In [2]:
import pandas as pd 

In [4]:
corpus_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/corpusTest.csv")
#corpus_test

In [3]:
corpus_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/corpusTrain.csv")
#corpus_train

In [5]:
corpus_test['Content'] = corpus_test['Content'].str.strip()
corpus_test['Content'] = corpus_test['Content'].str.lower()

corpus_train['Content'] = corpus_train['Content'].str.strip()
corpus_train['Content'] = corpus_train['Content'].str.lower()

In [6]:
spec_chars = ["!", '"', "#", "%", "&", "'", "(", ")", "*", "+", ",", "-", ".", "/", ":", ";", "<",
              "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "{", "|", "}", "~", "–"]

for char in spec_chars:
    corpus_test['Content'] = corpus_test['Content'].str.replace(char, '')

for char in spec_chars:
    corpus_train['Content'] = corpus_train['Content'].str.replace(char, '')

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 10000)
vectorizer.fit(corpus_test['Content'])
vectorized_test_content = vectorizer.transform(corpus_test['Content'])
print(vectorized_test_content.shape)

(5374, 9011)


In [9]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 10000)
vectorizer.fit(corpus_train['Content'])
vectorized_train_content = vectorizer.transform(corpus_train['Content'])
print(vectorized_train_content.shape)

(531990, 10000)


In [10]:
corpus_compined = pd.DataFrame(corpus_test['Content'].append(corpus_train['Content'], ignore_index = True)) 
#corpus_compined['Content']

In [11]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 10000)
vectorizer.fit(corpus_compined['Content'])
vectorized_combined_content = vectorizer.transform(corpus_compined['Content'])
print(vectorized_combined_content.shape)

(537364, 10000)


Exact Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
count = 0
i = 0
similar = []
res = cosine_similarity(vectorized_combined_content[0:5374], vectorized_combined_content[5374:100000])
for inner_res in res:
  for c in inner_res:
    if c > 0.8: 
      similar.append(i)
      count = count + 1
      break
  i = i + 1
print(count)

1098


In [ ]:
i = 0
res = cosine_similarity(vectorized_combined_content[0:5374], vectorized_combined_content[100000:200000])
for inner_res in res:
  for c in inner_res:
    if c > 0.8 and i not in similar: 
      similar.append(i)
      count = count + 1
      break
  i = i +1
print(count)

1914


In [ ]:
i = 0
res = cosine_similarity(vectorized_combined_content[0:5374], vectorized_combined_content[200000:300000])
for inner_res in res:
  for c in inner_res:
    if c > 0.8 and i not in similar: 
      similar.append(i)
      count = count + 1
      break
  i = i +1
print(count)

2176


In [ ]:
i = 0
res = cosine_similarity(vectorized_combined_content[0:5374], vectorized_combined_content[300000:400000])
for inner_res in res:
  for c in inner_res:
    if c > 0.8 and i not in similar: 
      similar.append(i)
      count = count + 1
      break
  i = i +1
print(count)

2381


In [ ]:
i = 0
res = cosine_similarity(vectorized_combined_content[0:5374], vectorized_combined_content[400000:])
for inner_res in res:
  for c in inner_res:
    if c > 0.8 and i not in similar: 
      similar.append(i)
      count = count + 1
      break
  i = i +1
print(count)

2604


In [ ]:
print("Duplicates identified with exact cosine: " + str(len(similar)))

Duplicates identified: 2196


Exact Jaccard Similarity

In [12]:
import numpy as np
from sklearn.metrics import jaccard_score, accuracy_score

In [13]:
vectorized_test_arr = vectorized_combined_content[0:5374].toarray()

In [14]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [30]:
for j in range(5374):
  c = jaccard_score(vectorized_test_arr[5373], vectorized_test_arr[j], average='macro')
  if c > 0.8: 
    count = count + 1

In [31]:
count = 0
temp = []
for i in range(5374, 6000):
  temp = vectorized_combined_content[i].toarray()
  for j in range(5374):
    c = jaccard_score(temp[0], vectorized_test_arr[j], average='macro')
    if c > 0.8: 
      count = count + 1
      break
print(count)

38


MinHash LSH

In [ ]:
!pip install --upgrade datasketch

In [ ]:
from datasketch import MinHash, MinHashLSH

In [ ]:
num_permutations = 32
count = 0
removed_items_indexes = []
for i in range(5374, 537364):
  set1 = set(corpus_train['Content'][i].split(" "))
  m1 = MinHash(num_perm=num_permutations)

  for d in set1:
      m1.update(d.encode('utf8'))

  for j in range(5374):
    set2 = set(corpus_test['Content'][j].split(" "))
    m2 = MinHash(num_perm=num_permutations)

    for d in set2:
        m2.update(d.encode('utf8'))

    # Create LSH index
    lsh = MinHashLSH(threshold=0.5, num_perm=num_permutations)
    lsh.insert("m2", m2)
    result = lsh.query(m1)
    if len(result) > 0: 
      count = count + 1
    else: 
      removed_ items_indexes.append(j)
print(count)

LSH-Cosine

In [ ]:
from sklearn.neighbors import LSHForest

In [ ]:
train_indexes_arr = []
for j in range(5374, 537364):
  temp = []
  for i in range(len(vectorized_combined_content[j].nonzero()[0])):
    temp.append(vectorized_combined_content[j].nonzero()[1][i])
  train_indexes_arr.append(temp)

In [ ]:
test_indexes_arr = []
for j in range(5374):
  temp = []
  for i in range(len(vectorized_combined_content[j].nonzero()[0])):
    temp.append(vectorized_combined_content[j].nonzero()[1][i])
  test_indexes_arr.append(temp)

In [ ]:
lshf = LSHForest()
lshf.fit(train_indexes_arr) 
distances, indices = lshf.kneighbors(test_indexes_arr, n_neighbors=2)

LSH-Jaccard

In [ ]:
#updated arrays, contain only the test rows with lsh similarity > 0.5
count = 0
for i in range(5374, 537364):
  for j in range(5374):
    c = jaccard_score(vectorized_train_arr[i], vectorized_test_arr[j])
    if c > 0.8: 
      count = count + 1
print(count)